In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
join = os.path.join
from tqdm import tqdm
from skimage import transform 
from torch.utils.data import Dataset, DataLoader
import torch.multiprocessing as mp
import sys
sys.path.append('../../modified_medsam_repo')
from segment_anything import sam_model_registry, build_sam_vit_b_multiclass
import random
from datetime import datetime
import pandas as pd
import nibabel as nib
import pickle   
from glob import glob

In [9]:
# build a SAM dataframe
dfo = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_constant_bbox.csv')
dfo_sam = dfo
dfo_sam['image_embedding_slice_path'] = [x.replace('pretrained_image_encoded_slices', 'SAM_image_encoded_slices') for x in dfo['image_embedding_slice_path']]
#dfo_sam.to_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/other_path_dfs/path_df_SAM_constant_bbox.csv', index=False)

In [15]:
# build MedSAM and UNet dataframes for Brats
dir_path = '/gpfs/data/luilab/karthik/MICCAI_BraTS2020_TrainingData'
pattern = os.path.join(dir_path, 'encoded_slices', '*', '*.npy')
listo = glob(pattern)
 
dicto = {'id': [], 'slice': [], 'image_embedding_slice_path': [], 'segmentation_slice_path': [], 'image_path': [], 'bbox_0':[], 'bbox_1':[], 'bbox_2':[], 'bbox_3':[]}
for x in listo:
    slice_num = int(x.split('/')[-1].split('.npy')[0])
    mri_id = str(x.split('/')[-2])
    image_embedding_slice_path = x
    image_path = os.path.join(dir_path, 'images', f'images{mri_id}_slice{slice_num}.png')
    segmentation_slice_path = os.path.join(dir_path, 'segmentations', mri_id, f'seg_{slice_num}.npy')

    dicto['id'].append(mri_id)
    dicto['slice'].append(slice_num)
    dicto['image_embedding_slice_path'].append(image_embedding_slice_path)
    dicto['segmentation_slice_path'].append(segmentation_slice_path)
    dicto['image_path'].append(image_path)

dicto['bbox_0'] = [0] * len(listo)
dicto['bbox_1'] = [0] * len(listo)
dicto['bbox_2'] = [256] * len(listo)
dicto['bbox_3'] = [256] * len(listo)

brats_medsam_df = pd.DataFrame(dicto)
brats_medsam_df = brats_medsam_df.sort_values(['id', 'slice']).reset_index(drop=True)
assert len(brats_medsam_df) == 50 * 256
brats_medsam_df.to_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/other_path_dfs/path_df_brats_constant_bbox.csv', index=False)

In [14]:
brats_medsam_df['id'] = brats_medsam_df['id'].astype(str)
brats_medsam_df['id']

0        001
1        001
2        001
3        001
4        001
        ... 
12795    050
12796    050
12797    050
12798    050
12799    050
Name: id, Length: 12800, dtype: object

In [35]:
# now unet dataframe
dicto_unet = {'id': dicto['id'], 'slice': dicto['slice'], 'img_slice_path': dicto['image_path'], 'segmentation_slice_path': dicto['segmentation_slice_path']}
brats_unet_df = pd.DataFrame(dicto_unet)
brats_unet_df = brats_unet_df.sort_values(['id', 'slice']).reset_index(drop=True)
#brats_unet_df.to_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/other_path_dfs/path_df_brats_unet.csv', index=False)